# Interpret VAE

This notebook interprets the weights and biases of a VAE to identify the genes most important to defining the learned clusters in latent space.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

import tensorflow as tf
from sklearn import preprocessing
from sklearn.manifold import TSNE
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Activation
from keras.utils import to_categorical

from framework.common.dataset import Dataset
from framework.keras.autoencoder import VariationalAutoencoder as VAE

np.random.seed(1013)
tf.set_random_seed(1013)

Using TensorFlow backend.


In [5]:
# Model directory
model_dir = "results/patel/Train1000gPatelGBMVAE/PatelGBMVAE_Final"

# File that lists the cluster associated with each cell
cluster_file = model_dir + "/consensus_clusters_k7.txt"
# Which column in the file to use as the clusters
cluster_col = "consensus_cluster"

dataset_file = "data/GSE57872_GBM/processed/gbm.1000g.centered.txt"
# GSE57872_GBM; GSE72056_Melanoma, GSE75688_Breast_Cancer
dataset_features_start_idx = 1
# Start indexes: melanoma = 1; gbm = 1; breast = 2

latent_space_file = model_dir + "/latent_representations.txt"
# What column in latent space file marks the end of 
# the features and the beginning of labels/other information
features_end_col_idx = -1

In [6]:
def load_dataset():
    dataset = pd.read_csv(dataset_file, sep="\t", header=0, index_col=0)
    features = dataset.iloc[:, dataset_features_start_idx:].values.astype(dtype=np.float64)
    cell_ids = dataset.index.values
    
    return dataset, cell_ids, features

def load_latent_space():
    df = pd.read_csv(latent_space_file, sep="\t", header=0, index_col=0)
    latent_space = df.iloc[:, 0:features_end_col_idx]
    
    return latent_space

def load_clusters():
    clusters_df = pd.read_csv(cluster_file, sep="\t", header=0, index_col=0)
    clusters = clusters_df.loc[:, cluster_col]
    return clusters

dataset, cell_ids, features = load_dataset()
latent_space = load_latent_space()

scaler = preprocessing.StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)

clusters = load_clusters()
clusters = clusters[cell_ids]

latent_space = latent_space.loc[cell_ids, :].values.astype(dtype=np.float64)

classes = np.unique(clusters.values)
print("Number of Clusters:", len(classes))

vae = VAE.restore(model_dir)

Number of Clusters: 7


In [ ]:
get_mean_log_var = K.function([vae.autoencoder_model.layers[0].input, K.learning_phase()],
                              [vae.autoencoder_model.layers[5].output,
                               vae.autoencoder_model.layers[6].output])

In [ ]:
latent_mean, latent_log_var = tuple(get_mean_log_var([features_scaled, 0]))

In [ ]:
# Fit a t-SNE model to the features for visualization purposes
def fit_tsne(features):
    tsne_model = TSNE(n_components=2, init='pca', random_state=0, perplexity=30)
    return tsne_model.fit_transform(features)

tsne_output = fit_tsne(latent_mean)

In [ ]:
tsne_plot_labels = list(clusters)
plt.scatter(tsne_output[:, 0], tsne_output[:, 1], 
            c=tsne_plot_labels, cmap=plt.cm.get_cmap(
                "rainbow", len(np.unique(tsne_plot_labels))))
plt.colorbar(ticks=np.unique(tsne_plot_labels))
plt.clim(np.min(tsne_plot_labels)-0.5, np.max(tsne_plot_labels) + 0.5)
plt.show()

In [ ]:
vae.encoder_model.summary()

In [7]:
mean_encoder_model = Model(vae.encoder_model.layers[0].input, vae.encoder_model.layers[5].output)
mean_encoder_model.trainable = False
clustering_output = Dense(7, activation="linear")(mean_encoder_model(vae.x))
clustering_output = Activation("softmax")(clustering_output)
clustering_model = Model(vae.x, clustering_output)

In [8]:
clustering_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
x (InputLayer)               (None, 1000)              0         
_________________________________________________________________
model_4 (Model)              (None, 10)                1438010   
_________________________________________________________________
dense_6 (Dense)              (None, 7)                 77        
_________________________________________________________________
activation_1 (Activation)    (None, 7)                 0         
Total params: 1,438,087
Trainable params: 77
Non-trainable params: 1,438,010
_________________________________________________________________


In [9]:
clustering_model.compile(loss="categorical_crossentropy", optimizer="adam",
                         metrics=["accuracy"])

In [10]:
clustering_model.fit(features_scaled, to_categorical(clusters.values - 1), epochs=100, batch_size=50)

Epoch 1/100
430/430 [==============================] - 0s - loss: 1.7826 - acc: 0.3093     
Epoch 2/100
430/430 [==============================] - 0s - loss: 1.7293 - acc: 0.3279     
Epoch 3/100
430/430 [==============================] - 0s - loss: 1.6773 - acc: 0.3581     
Epoch 4/100
430/430 [==============================] - 0s - loss: 1.6297 - acc: 0.3837     
Epoch 5/100
430/430 [==============================] - 0s - loss: 1.6074 - acc: 0.4070     
Epoch 6/100
430/430 [==============================] - 0s - loss: 1.5668 - acc: 0.4465     
Epoch 7/100
430/430 [==============================] - 0s - loss: 1.4956 - acc: 0.4628     
Epoch 8/100
430/430 [==============================] - 0s - loss: 1.4786 - acc: 0.4884     
Epoch 9/100
430/430 [==============================] - 0s - loss: 1.4198 - acc: 0.5419     
Epoch 10/100
430/430 [==============================] - 0s - loss: 1.4068 - acc: 0.5140     - ETA: 0s - loss: 1.4249 - acc: 0.505
Epoch 11/100
430/430 [====================

430/430 [==============================] - 0s - loss: 0.3520 - acc: 0.9302     
Epoch 89/100
430/430 [==============================] - 0s - loss: 0.3604 - acc: 0.9326     
Epoch 90/100
430/430 [==============================] - 0s - loss: 0.3548 - acc: 0.9326     
Epoch 91/100
430/430 [==============================] - 0s - loss: 0.3429 - acc: 0.9279     
Epoch 92/100
430/430 [==============================] - 0s - loss: 0.3496 - acc: 0.9279     
Epoch 93/100
430/430 [==============================] - 0s - loss: 0.3416 - acc: 0.9326     
Epoch 94/100
430/430 [==============================] - 0s - loss: 0.3441 - acc: 0.9233     
Epoch 95/100
430/430 [==============================] - 0s - loss: 0.3421 - acc: 0.9302     
Epoch 96/100
430/430 [==============================] - 0s - loss: 0.3324 - acc: 0.9279     
Epoch 97/100
430/430 [==============================] - 0s - loss: 0.3263 - acc: 0.9395     
Epoch 98/100
430/430 [==============================] - 0s - loss: 0.3183 - acc: 0.

In [11]:
# R = latent_mean
# R[:, 1:] = 0
pred_clusters = K.function([clustering_model.layers[0].input, K.learning_phase()],
                           [clustering_model.layers[2].output])([features_scaled, 0])[0]
R = np.maximum(pred_clusters, 0)

In [12]:
# Propagate back through clustering model dense layer
last_layer_W = clustering_model.layers[2].get_weights()[0]
last_layer_W_sq = np.square(np.expand_dims(last_layer_W, 0))

last_layer_N = last_layer_W_sq / np.expand_dims(np.sum(last_layer_W_sq, 1), 1)
R = np.sum((last_layer_N * np.expand_dims(R, 1)), 2)

In [13]:
# Propagate back through z_mean layer
# z_mean_input = K.function([vae.encoder_model.layers[0].input, K.learning_phase()],
#                           [vae.encoder_model.layers[4].output])([features_scaled, 0])
z_mean_W = vae.encoder_model.layers[5].get_weights()[0]
z_mean_W_sq = np.square(np.expand_dims(z_mean_W, 0))

z_mean_N = z_mean_W_sq / np.expand_dims(np.sum(z_mean_W_sq, 1), 1)
R = np.sum((z_mean_N * np.expand_dims(R, 1)), 2)

In [14]:
# Propagate back through 2nd encoder layer
# layer_2_input = K.function([vae.encoder_model.layers[0].input, K.learning_phase()],
#                            [vae.encoder_model.layers[2].output])([features_scaled, 0])
layer_2_W = vae.encoder_model.layers[3].get_weights()[0]
layer_2_W_sq = np.square(np.expand_dims(layer_2_W, 0))

layer_2_N = layer_2_W_sq / np.expand_dims(np.sum(layer_2_W_sq, 1), 1)
R = np.sum((layer_2_N * np.expand_dims(R, 1)), 2)

In [15]:
# Propagate back through 1st encoder layer
# layer_1_input = features_scaled
layer_1_W = vae.encoder_model.layers[1].get_weights()[0]
layer_1_W_sq = np.square(np.expand_dims(layer_1_W, 0))

layer_1_N = layer_1_W_sq / np.expand_dims(np.sum(layer_1_W_sq, 1), 1)
R = np.sum((layer_1_N * np.expand_dims(R, 1)), 2)

In [16]:
result = features_scaled * R

In [17]:
dataset.columns[1:].values[result[0] > 0]

array(['AGT', 'METTL7B', 'EFEMP1', 'POSTN', 'SCG2', 'TNC', 'SERPINA3',
       'EGFR', 'LANCL2', 'SLC2A3', 'ALDOC', 'BCAN', 'KLHL4', 'PLS3',
       'DUSP6', 'PSAT1', 'BCAT1', 'MEG3', 'BTG2', 'TXNIP', 'KLF10',
       'MAP1A', 'BHLHE40', 'WARS', 'TPP1', 'CXCR7', 'NEDD9', 'TFRC',
       'SGK1', 'KPNA2', 'MCM7', 'ID2', 'GLB1', 'QARS', 'VCAN', 'GSS',
       'PRMT5', 'CTSA', 'MIR198', 'EZR', 'PLTP', 'C5orf62', 'AHCYL1',
       'CPVL', 'TSPAN6', 'LHFPL3', 'RFX4', 'EDNRB', 'ELOVL2', 'NCALD',
       'TSR1', 'TMEM43', 'TCTN3', 'FKBP5', 'IVNS1ABP', 'UBR7', 'PLOD1',
       'HERPUD1', 'ANXA7', 'PRCP', 'CLDN12', 'TCP1', 'SAT1', 'HMG20A',
       'PRNP', 'AARS', 'MEIS1', 'JAM3', 'SAE1', 'TSC22D1', 'CARS',
       'ALDH9A1', 'C19orf48', 'SEC61A1', 'HIF1A', 'ARID5B', 'ZNF146',
       'CTSL1', 'ARCN1', 'FAM126A', 'PLEKHB1', 'AASDHPPT', 'ACO2', 'HEXA',
       'PFN2', 'PMP2', 'TSC22D3', 'PCIF1', 'TRIM22', 'APEH', 'STOML2',
       'GDAP1', 'RSU1', 'DNAJB1', 'DDAH1', 'IL1RAP', 'ALDH6A1', 'ALDH1L1',
       'SEC

In [20]:
np.min(result)

-0.027032083010134298